## Bibliotecas

In [3]:
from sentence_transformers import SentenceTransformer
import spacy
import torch
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClusterig

## Pré-Processamento

In [ ]:
with open('./mergedfile.txt', 'r', encoding = "utf8") as arquivo:
    texto = arquivo.read().replace("-\n", "").replace("\n", " ")

# Carregue o modelo
nlp = spacy.load("pt_core_news_lg")

# Processamento do texto
doc = nlp(texto[0:1000000])

sentences = list(sent.text for sent in doc.sents)

### Verificar disponibilidade de placa gráfica

In [5]:
torch.cuda.is_available()

In [6]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

True

## Modelagem

### Codificação

In [12]:
# Codifique as sentenças usando o modelo
embeddings = model.encode(sentences, show_progress_bar=True, convert_to_tensor=True, device=device)

Batches: 100%|██████████| 249/249 [00:44<00:00,  5.55it/s]


### Teste

In [36]:
new_sentence = "A amazôniza azul não possui extensão de 30000 km^2"
new_embedding = model.encode([new_sentence], convert_to_tensor=True, device=device)

In [37]:
np_embeddings = embeddings.cpu().numpy()
np_input = new_embedding.cpu().numpy()

In [38]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Calcule a similaridade de cosseno entre a nova sentença e as sentenças existentes
similarities = util.pytorch_cos_sim(new_embedding, embeddings)[0]

# Encontre os índices das sentenças mais similares (N sentenças)
N = 5
most_similar_indices = (-similarities).argsort()[:N]

In [39]:
for idx in most_similar_indices:
    print(sentences[idx])

A extensão inicial da Amazônia Azul correspondem a 3,6 milhões de km?
Quantos quilômetros quadrados ocupa a Amazônia Azul?
Quantos metros quadrados a Amazônia Azul  ocupa?  
Qual é a extensão territorial da Amazônia Azul?
A expansão da Amazônia Azul: a plataforma continental do Brasil além das 200 milhas náuticas.


In [ ]:
#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")